# Test Millera-Rabina

#### Autorzy:
- Mateusz Serek 
- Weronika Sadoch

### Załadujmy niezbędne biblioteki:

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import random
from ipywidgets import interactive
import ipywidgets as widgets

## Wprowadzenie

Test pierwszości Millera-Rabina został opracowany w 1975 roku przez Michaela Rabina na podstawie wcześniejszych prac Gary'ego Millera. Pozwala on sprawdzić, czy dana liczba naturalna $n$ jest liczbą złożoną czy prawdopodobnie pierwszą.

Metoda ta opiera się na deterministycznym algorytmie Millera, którego poprawność jest warunkowa i zależy od nieudowodnionej uogólnionej hipotezy Riemanna. Michael O. Rabin zmodyfikował ten algorytm, przekształcając go w test probabilistyczny i udowodnił jego poprawność w tej formie.

Algorytm stosuje się wyłącznie do liczb nieparzystych, ponieważ liczby parzyste (z wyjątkiem dwójki) są w oczywisty sposób liczbami złożonymi.


## Twierdzenia
Test w głównej mierze opiera się na twierdzeniu Eulera oraz własności ciągu Millera-Rabina:
### Twierdzenie Eulera
Niech $n$ będzie liczbą pierwszą, $a$ liczbą naturalną taką, że $NWD(a,n)=1$. Wtedy zachodzi:
$ a^{n-1} \equiv 1  \pmod n$

Z twierdzenia wynika, że jeśli $n$ jest liczbą pierwszą, to dla dowolnego $a$, które jest względnie pierwsze z $n$, spełniona jest równość:
$ a^{n-1} \equiv 1 \pmod n$. 

Jeżeli równość nie zostanie spełniona, to liczba $n$ jest liczbą złożoną.

### Ciąg Millera Rabina

Niech $p$ będzie nieparzystą liczbą pierwszą zapisaną jako $p = 1+2^sd$, gdzie $d$ jest liczbą nieparzystą. 

Wtedy dla dowolnej liczby naturalnej $a ∈ <2, p-2>$ tworzymy ciąg Millera-Rabina: 
    $a^d, a^{2d}, a^{4d}, …, a^{2^{r-1}}, a^{2^r} \pmod p$
, który kończy się liczbą $1$. 

Dodatkowo, jeśli $a^d \not\equiv 1 \pmod p$ do $1$, to wyraz ciągu Millera-Rabina, który bezpośrednio poprzeda $1$, jest równy $p-1$.

## Cechy 
Test Millera-Rabina jest testem probalistycznym, co oznacza, że metoda jest szybka, ale może zwrócić błędną odpowiedź z pewnym prawdopodobieństwem, które możemy kontrolować zmieniając liczbe prób.

Po wykonaniu testu dostajemy jedną z dwóch odpowiedzi:
- `FALSE` – liczba **złożona** (z prawdopodobieństwem 100%)
- `TRUE` – liczba **prawdopodobnie pierwsza** (ryzyko błędu maleje wykładniczo wraz z liczbą prób).


Jeśli dla pewnej bazy $a$ test zwróci `FALSE`, oznacza to, że liczba złamała warunki wynikające z twierdzenia Eulera. Oznacza to, że mamy **pewność**, że jest to liczba **złożona**.



## Algorytm - wersja podstawowa
Dane są $n >= 5$
1. Szukamy takich liczb naturalnych $s, d$: $s \neq d$, gdzie $s$ jest nieparzyste oraz $n - 1  = 2 ^{s}$ * $ d$.
2. Wybierzmy liczbę losową $a$ taką, że $1 < a < n$
3. Niech $b = a ^{d}$. Jeżeli $b \equiv  1$ (mod $n$), zwróć prawdę.
4. W przeciwnym wypadku jeżeli $b^{2^{r}} \equiv -1$ (mod $n$)  $\forall r$ $1 \le  r \le  s-1$, zwróć prawdę.
5. W przeciwnym wypadku zwróć fałsz

Zwrócona prawda oznacza, że liczba $n$ jest prawdopodobnie pierwsza, w przypadku fałszu liczba $n$ napewno nie jest liczbą pierwszą.

## Prawdopodobieństwo oraz ulepszenie testu

Gdy liczba $p$ przejdzie test, oznacza to, że albo jest liczbą pierwszą albo silnie pseudopierwszą względem podstawy $a$.

Test Millera–Rabina może błędnie uznać liczbę złożoną za pierwszą, jednak prawdopodobieństwo wynosi $\frac{1}{4}$, a dla $n$ przebiegów przy różnych bazach $a$ prawdopodobieństwo błędu spada do $(\frac{1}{4})^n$. Już dwadzieścia testów daje szansę $1 : 1099511627776$, że liczba złożona $p$ zostanie potraktowana jako liczba pierwsza. 
Można zatem stwierdzić, że im więcej wykonamy losowych testów, tym większa jest pewność poprawności wyniku.

### Wykonaj poniższy kod aby zobaczyć wizualizację

In [6]:
def show_probability(N):
    n = np.arange(N, N + 5)
    y = (1/4) ** n

    plt.figure(figsize = (10, 8))
    plt.plot(n, y, 'o-', color='blue', label = '0.25 ** n')
    plt.title('Prawdopodobieństwo fałszywej odpowiedzi dla liczby prób')
    plt.xlabel('Liczba prób (n)')
    plt.ylabel('Prawdopodobieństwo')

    plt.grid(True)
    plt.legend()
    plt.xticks(n)
    plt.show()

interactive(show_probability, N = widgets.IntSlider(min = 1, max = 6, step = 1, value = 1, description = 'Zmiana skali'))

interactive(children=(IntSlider(value=1, description='Zmiana skali', max=6, min=1), Output()), _dom_classes=('…

Zmieniając skalę wykresu widzimy, jak szybko zmiejsza się prawdopodobieństwo niepoprawnej odpowiedzi.

## Implementacja Testu Millera-Rabina w SageMath (Python)

In [8]:
def miller_rabbin(n: int, k: int) -> bool: 
    #n - testowana liczba
    #k - ilość prób 

    def power_mod(a: int, b: int, c: int) -> int: #funkcja pomocnicza do obliczenia potęgi modulo
        result = 1
        for i in range(b):
            result *= a 
            result %= c 
        return result
    
    if n < 5:
        raise ValueError(f"number = {n} cannot be below 5")
        
    if k < 0:
        raise ValueError(f"number of tests = {k} cannot be below 0")
        
    if n % 2 == 0:
        return False
    
    d = n - 1
    s = 0
    
    while d % 2 == 0: #Krok 1.
        s += 1
        d //= 2
        
    for p in range(k): # Po ulepszeniu algorytmu wykonywane jest wiele prób. 
        a = random.randint(2, n) #Krok 2.
        b = power_mod(a, d, n)
        
        for q in range(s): 
            y = (b * b) % n
            
            if y == 1 and b != 1 and b != n - 1: # -1 (mod n) = n - 1 (mod n), warunek z kroku 4.
                return False
            
            b = y
            
        if y != 1: # W ulepszonej wersji algorytmu warunek z kroku 3. jest sprawdzany na końcu iteracji.
            return False
        
    return True  # Jeśli żadna próba nie wykluczy pierwszości, zwrócona zostanie prawda

### Sprawdźmy działanie algorytmu.

In [9]:
numbers = [random.randint(10 ** 5, 10 ** 6) for i in range(20)]
for i in numbers:
    result = miller_rabbin(i, 9) # wykonywane 9 prób dla kazdej liczby.
    if result:
        print(f"Liczba {i} prawdopodobnie jest pierwsza")
    else:
        print(f"Liczba {i} na pewno nie jest pierwsza")

Liczba 550034 na pewno nie jest pierwsza
Liczba 164060 na pewno nie jest pierwsza
Liczba 255745 na pewno nie jest pierwsza
Liczba 154800 na pewno nie jest pierwsza
Liczba 422453 prawdopodobnie jest pierwsza
Liczba 667676 na pewno nie jest pierwsza
Liczba 300128 na pewno nie jest pierwsza
Liczba 400030 na pewno nie jest pierwsza
Liczba 918363 na pewno nie jest pierwsza
Liczba 138611 na pewno nie jest pierwsza
Liczba 156421 prawdopodobnie jest pierwsza
Liczba 343758 na pewno nie jest pierwsza
Liczba 579848 na pewno nie jest pierwsza
Liczba 460279 na pewno nie jest pierwsza
Liczba 833725 na pewno nie jest pierwsza
Liczba 967997 na pewno nie jest pierwsza
Liczba 330945 na pewno nie jest pierwsza
Liczba 131801 na pewno nie jest pierwsza
Liczba 709850 na pewno nie jest pierwsza
Liczba 770601 na pewno nie jest pierwsza


Prawdopodobieństwo nieprawidłowej odpowiedzi dla liczb, które zostały uznane za pierwsze to:
$$(\frac{1}{4})^9 \approx 3,8 * 10^{-6}$$
gdzie $9$ to liczba wykonanych prób.


## Złożoność obliczeniowa

Dla jednej bazy obliczenia opierają się na szybkim potęgowaniu modulo: 
$ O(log \text{ }d)$.

W najgorszym przypadku algorytm wykonuje maksymalnie $s$ kroków sprawdzających, co prowadzi do złożoności rzędu $O(log^2 n)$

Aby zwiększyć prawdopodobieństwo poprawnego wyniku, test wykonuje się wielokrotnie dla różnych baz.  
Dla $a$ niezależnych baz złożoność rośnie liniowo względem ich liczby i wynosi:  
$ O(a \cdot log^2 n)$

W praktyce algorytm okazuje się bardzo wydajny podczas  sprawdzania pierwszości dużych liczb.

## Zastosowania

Test wykorzystujemy wszędzie tam, gdzie potrzebne są duże liczby pierwsze, np. w:
- kryptografii (generowanie kluczy RSA),
- protokołach bezpieczeństwa,
- systemach szyfrowania danych,
- losowaniu dużych liczb pierwszych.

### Źródła
https://wstein.org/ent/ent.pdf <br>
https://en.wikipedia.org/wiki/Miller–Rabin_primality_test <br>
http://www.algorytm.org/algorytmy-arytmetyczne/test-pierwszosci-test-millera-rabina.html <br>
https://eduinf.waw.pl/inf/alg/001_search/0019.php